In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import itertools
import warnings

In [21]:
def mmn_queueing_redundancy_dos(n,d,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the redundancy method, the user needs to provide
    the number of servers (n), the number of copies to send (d), arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    Using the delete on start method.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    d:                the number of copies to send
    arrival_rate:     exponential rate of arrival
    service_rate:     exponential rate of service time
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)
        np.random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    if d < 1 or d > n:
        raise Exception("d needs to be in the range 1 <= d <= n")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int or list type for service rate")
    
    # Simulating the Redundancy queues
    redun_server = np.zeros(n,dtype=int)
    for i in range(1,n+1):
        exec(f'redun_queue_{i} = []')
    copyhistory = pd.DataFrame(np.zeros(d,dtype=int)).T
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1,dtype=int)  # keep track of the number customers
    no_completed_service = np.zeros(n+1,dtype=int)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            copy_list = np.random.choice(np.arange(n),size=d,replace=False)
            arrival_label += 1
            
            # If all servers are occupied
            if sum(redun_server[copy_list]==0) == 0:
                event_calendar[-1,queue_col[copy_list]] += 1
                event_calendar[-1,total_col[copy_list]] += 1
                for i in copy_list:
                    eval(f'redun_queue_{i+1}').append(arrival_label)
                new_history = pd.DataFrame(copy_list).T
                new_history.index = [f"{arrival_label}"]
                copyhistory = copyhistory.append(new_history)
            
            # If only one server is available
            elif sum(redun_server[copy_list]==0) == 1:
                avail_server = copy_list[np.where(redun_server[copy_list]==0)[0][0]]
                event_calendar[-1,table_col[avail_server]] += 1
                event_calendar[-1,avail_col[avail_server]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[avail_server])
                event_calendar[-1,total_col[avail_server]] += 1
                redun_server[avail_server] = arrival_label
                no_customers[avail_server] += 1
                
            # If more than one servers are available
            else:
                avail_server = random.choice(copy_list[np.where(redun_server[copy_list]==0)[0]])
                event_calendar[-1,table_col[avail_server]] += 1
                event_calendar[-1,avail_col[avail_server]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[avail_server])
                event_calendar[-1,total_col[avail_server]] += 1
                redun_server[avail_server] = arrival_label
                no_customers[avail_server] += 1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = 0
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_completion])
                redun_server[server_completion] = eval(f'redun_queue_{server_completion+1}')[0]
                for i in copyhistory.loc[f"{redun_server[server_completion]}"]:
                    event_calendar[-1,queue_col[i]] -= 1
                    event_calendar[-1,total_col[i]] -= 1
                    eval(f'redun_queue_{i+1}').remove(redun_server[server_completion])
                copyhistory = copyhistory.drop(f"{redun_server[server_completion]}")
                no_customers[server_completion] += 1
                
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time (有点问题，没完成的不应该加入，long-term error可忽略)
    awt_data = ((time3 @ aql_server)/no_customers).round(3)
    awt = pd.DataFrame(awt_data).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time (有点问题，没完成的不应该加入，long-term error可忽略)
    act_data = ((time3 @ ans_server)/no_customers).round(3)
    act = pd.DataFrame(act_data).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]
    
    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "mu": mu,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat


In [40]:
aaa = mmn_queueing_redundancy_dos(5,3,20,[4,7,2,8,5],simulation_time=20,simuseed=8)
bbb = mmn_queueing_redundancy_doc(5,3,20,[4,7,2,8,5],simulation_time=20,simuseed=8)

In [43]:
aaa[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization,Server5 Utilization
0,0.801,0.639,0.889,0.604,0.752


In [44]:
bbb[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization,Server5 Utilization
0,0.814,0.769,0.813,0.739,0.811


In [46]:
aaa[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Server5 Average number in the system,Grand Average number in the system
0,1.72,1.371,1.691,1.44,1.74,7.962


In [47]:
bbb[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Server5 Average number in the system,Grand Average number in the system
0,2.117,1.916,2.182,1.953,2.183,10.352


In [48]:
aaa[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Server5 Average queue length,Grand Average queue length
0,0.919,0.732,0.802,0.836,0.987,4.277


In [49]:
bbb[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Server5 Average queue length,Grand Average queue length
0,1.303,1.147,1.369,1.214,1.372,6.405


In [50]:
aaa[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Server5 Average waiting time,Grand Average waiting time
0,0.292,0.151,0.411,0.158,0.278,0.227


In [51]:
bbb[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Server5 Average waiting time,Grand Average waiting time
0,0.383,0.207,0.856,0.213,0.312,0.31


In [53]:
aaa[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Server5 Average cycle time,Grand Average cycle time
0,0.546,0.283,0.867,0.272,0.49,0.424


In [52]:
bbb[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Server5 Average cycle time,Grand Average cycle time
0,0.623,0.345,1.364,0.343,0.496,0.501


In [55]:
aaa[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,63,97,39,106,71,376
Completed services,62,96,38,105,70,371


In [54]:
bbb[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,68,111,32,114,88,413
Completed services,64,108,29,112,85,398


In [58]:
def mmn_queueing_redundancy_doc(n,d,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the redundancy method, the user needs to provide
    the number of servers (n), the number of copies to send (d), arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    Using the delete on completion method.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    d:                the number of copies to send
    arrival_rate:     exponential rate of arrival
    service_rate:     exponential rate of service time
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)
        np.random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    if d < 1 or d > n:
        raise Exception("d needs to be in the range 1 <= d <= n")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int or list type for service rate")
    
    # Simulating the Redundancy queues
    redun_server = np.zeros(n,dtype=int)
    for i in range(1,n+1):
        exec(f'redun_queue_{i} = []')
    copyhistory = pd.DataFrame(np.zeros(d,dtype=int)).T
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1,dtype=int)  # keep track of the number customers
    no_completed_service = np.zeros(n+1,dtype=int)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            copy_list = np.random.choice(np.arange(n),size=d,replace=False)
            arrival_label += 1
            new_history = pd.DataFrame(copy_list).T
            new_history.index = [f"{arrival_label}"]
            copyhistory = copyhistory.append(new_history)
            
            for i in copy_list:
                no_customers[i] += 1
                
                # If the server is idle
                if event_calendar[-1,table_col[i]] == 0:
                    event_calendar[-1,table_col[i]] += 1
                    event_calendar[-1,avail_col[i]] = event_calendar[-1,0]+random.expovariate(mu[i])
                    event_calendar[-1,total_col[i]] += 1
                    redun_server[i] = arrival_label
                    
                # If the server is occupied
                elif event_calendar[-1,table_col[i]] == 1:
                    event_calendar[-1,queue_col[i]] += 1
                    event_calendar[-1,total_col[i]] += 1
                    eval(f'redun_queue_{i+1}').append(arrival_label)
        
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            job_completion = redun_server[server_completion]
            
            # For the completion server
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = 0
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = eval(f'redun_queue_{server_completion+1}')[0]
                eval(f'redun_queue_{server_completion+1}').remove(redun_server[server_completion])
            
            # For the other servers in the copylist
            for i in copyhistory.loc[f"{job_completion}"]:
                if i != server_completion:
                    no_customers[i] -= 1
                
                    # If the completed job is undertaken
                    if redun_server[i] == job_completion:
                        
                        # If there is no queue behind
                        if event_calendar[-1,queue_col[i]] == 0:
                            event_calendar[-1,table_col[i]] -= 1
                            event_calendar[-1,avail_col[i]] = math.nan
                            event_calendar[-1,total_col[i]] -= 1
                            redun_server[i] = 0

                        # Else if there is a queue behind
                        elif event_calendar[-1,queue_col[i]] > 0:
                            event_calendar[-1,avail_col[i]] = event_calendar[-1,0] + \
                                                                    random.expovariate(mu[i])
                            event_calendar[-1,queue_col[i]] -= 1
                            event_calendar[-1,total_col[i]] -= 1
                            redun_server[i] = eval(f'redun_queue_{i+1}')[0]
                            eval(f'redun_queue_{i+1}').remove(redun_server[i])
                    
                    # Else if the completed job is not undertaken
                    elif redun_server[i] != job_completion:

                        event_calendar[-1,queue_col[i]] -= 1
                        event_calendar[-1,total_col[i]] -= 1
                        eval(f'redun_queue_{i+1}').remove(job_completion)

            copyhistory = copyhistory.drop(f"{job_completion}")


                
                
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time (有点问题，没完成的不应该加入，long-term error可忽略)
    awt_data = ((time3 @ aql_server)/no_customers).round(3)
    awt = pd.DataFrame(awt_data).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time (有点问题，没完成的不应该加入，long-term error可忽略)
    act_data = ((time3 @ ans_server)/no_customers).round(3)
    act = pd.DataFrame(act_data).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]
    
    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "mu": mu,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat


In [59]:
mmn_queueing_redundancy_doc(5,3,20,[4,7,2,8,5],simulation_time=20,simuseed=8)[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server4,Server4 Available Time,Server4 Queue,Server4 Total,Server5,Server5 Available Time,Server5 Queue,Server5 Total,Grand Queue,Grand Total
1,0.000000,0.012855,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,0.012855,0.176753,20.0,0.0,NaN,0.0,0.0,1.0,0.025574,0.0,...,0.0,NaN,0.0,0.0,1.0,0.039866,0.0,1.0,0.0,3.0
3,0.025574,0.176753,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
4,0.176753,0.190967,20.0,0.0,NaN,0.0,0.0,1.0,0.210319,0.0,...,1.0,0.305110,0.0,1.0,1.0,1.585821,0.0,1.0,0.0,3.0
5,0.190967,0.221696,20.0,0.0,NaN,0.0,0.0,1.0,0.210319,0.0,...,1.0,0.305110,1.0,2.0,1.0,1.585821,1.0,2.0,2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,19.903061,20.103406,20.0,1.0,20.163887,6.0,7.0,1.0,20.010636,4.0,...,1.0,19.918434,2.0,3.0,1.0,19.945632,3.0,4.0,19.0,24.0
800,19.918434,20.103406,20.0,1.0,20.163887,5.0,6.0,1.0,19.954293,3.0,...,1.0,20.269666,1.0,2.0,1.0,19.945632,3.0,4.0,16.0,21.0
801,19.945632,20.103406,20.0,1.0,20.275227,4.0,5.0,1.0,19.954293,3.0,...,1.0,20.269666,1.0,2.0,1.0,20.285806,2.0,3.0,13.0,18.0
802,19.954293,20.103406,20.0,1.0,20.275227,3.0,4.0,1.0,20.007370,2.0,...,1.0,20.269666,1.0,2.0,1.0,20.285806,2.0,3.0,10.0,15.0
